In [2]:
import kagglehub
import pandas as pd
import os

ModuleNotFoundError: No module named 'kagglehub'

In [ ]:
path = kagglehub.dataset_download("n2cholas/competitive-pokemon-dataset")
print("Ruta de descarga:", path)
print("Archivos disponibles:", os.listdir(path))

df_pokemon = pd.read_csv(os.path.join(path, "pokemon-data.csv"), sep=';')
df_moves = pd.read_csv(os.path.join(path, "move-data.csv"))

print("\nPokemon dataset:")
display(df_pokemon.head())

print("\nMoves dataset:")
display(df_moves.head())

ModuleNotFoundError: No module named 'kagglehub'